In [32]:
suppressWarnings(require('pkgmaker',quietly = T))
require('plyr',quietly = T)
l_ply(c('dplyr',
        'data.table',
#         'jsonlite', 
#         'rjson',
#         'httr',
#         'DEqMS', 
#         'pcaMethods',
#         'RMariaDB',
#         'parmigene',
#         'matrixTests',
#         'plotly',
        'tictoc',
        'tidyr',  
        'reshape2',
        'factoextra',
        'kableExtra',
        'parallel',
        'doParallel',
        'scales',
        'StatMatch',
        'lattice',
        'utils',
        'missMDA',
        'missForest',
        'VIM',
        'pacman',
        'hablar',
        'tibble'
       ), function(pkg) {
          invisible(capture.output(irequire(pkg, quiet = T, autoinstall = T)))
        })

## Protein_intensity table

From /general_analysis/DataBaseUpload/NGS/Proteomics/bin/PhosphoProteomics_PreProccess.html:

Outlayiers: very low/high values (0.0001% and 99.9999%) were floored to these ranges. This believed to be an error and unlikely to be real value.

Possible reason: the original data have Inf. The R language calculated it to some extreme values.

In [13]:
# from /general_analysis/DataBaseUpload/NGS/Proteomics/bin/PhosphoProteomics_PreProccess.html
# `Counts` are just 2 to the power of Intensity. This is for DE analysis where counts are needed
# note: without filtering, the dimension is 1330824 rows
intensity <- read.csv("../proteomics/data/Protein_intensity.csv") 
# %>% filter(Intensity > -3.0) # remove values that are too small - think if this is necessary

In [12]:
head(intensity, 2)
print(dim(intensity))

id,Protein,ProteinGroup,ProteinGroupName,ProteinGroupId,Organism,Sample,Intensity,counts,hgnc_symbol
45243,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0158,15.503,NULL,NULL
45244,sp|A0A087WV62|TVB16_HUMAN,sp|A0A087WV62|TVB16_HUMAN,TVB16_HUMAN,A0A087WV62,NULL,CTG-0159,14.932,NULL,NULL


[1] 1330824      10


In [2]:
# intensity[intensity$Sample=='CTG-0166',]

In [3]:
# # check a case where there are repeats
# # The intensity was not calculated differently between GAL3A and GAL3B
# x <- intensity[intensity$Protein=='sp|A0A0B4J2D5|GAL3B_HUMAN',]
# x[x$Sample=='CTG-0158',]

In [19]:
int_mtx <- intensity %>% 
    select(c('ProteinGroup', 'Sample', 'Intensity'))  %>% 
    pivot_wider(names_from = Sample, values_from = Intensity, values_fn = mean) %>% 
#     pivot_wider(names_from = Sample, values_from = Intensity)
    column_to_rownames('ProteinGroup') %>% 
    na.omit %>% 
    t
head(int_mtx, 2)
print(dim(int_mtx))

,sp|A0AVT1|UBA6_HUMAN,sp|O00170|AIP_HUMAN,sp|O00299|CLIC1_HUMAN,sp|O00429|DNM1L_HUMAN,sp|O00483|NDUA4_HUMAN,sp|O00571|DDX3X_HUMAN,sp|O14828|SCAM3_HUMAN,sp|O14929|HAT1_HUMAN,sp|O15143|ARC1B_HUMAN,sp|O15145|ARPC3_HUMAN,...,sp|Q9Y230|RUVB2_HUMAN,sp|Q9Y285|SYFA_HUMAN,sp|Q9Y2B0|CNPY2_HUMAN,sp|Q9Y2X3|NOP58_HUMAN,sp|Q9Y383|LC7L2_HUMAN,sp|Q9Y4L1|HYOU1_HUMAN,sp|Q9Y5K5|UCHL5_HUMAN,sp|Q9Y5M8|SRPRB_HUMAN,sp|Q9Y5X3|SNX5_HUMAN,sp|Q9Y678|COPG1_HUMAN
CTG-0158,16.977,17.204,19.893,16.449,17.447,17.986,17.925,18.063,15.901,19.330,...,18.845,16.672,18.265,18.357,18.928,20.587,17.358,16.812,17.509,18.347
CTG-0159,16.611,16.658,20.509,16.127,16.884,18.454,14.645,16.471,16.668,19.493,...,19.536,17.021,18.243,18.290,18.476,18.608,17.688,17.074,16.614,18.666


[1] 317 295


In [9]:
rm(intensity)  # release memory since using int_mtx from here on

The intensity is already in log-scale because it has negative values.

## Standardize the data before imputation

*Standardizing the data makes it more interpretable for the errors after imputation.*

Missing values in proteomic data can be generally characterized into missing at random (MAR) and missing not at random (MNAR). 
+ MAR missing values mostly result from technical limitations and stochastic fluctuations in an abundance-independent manner.
+ MNAR missing values are more abundance-dependent that can be explained by the measurability of the corresponding peptides. 

Missing values in proteomic data are a mixture of MAR and MNAR. Although the real proportion is difficult to determine, it is believed that MNAR plays a dominant role in producing missing values.

In [20]:
int_z <- int_mtx %>%
    sweep(2, apply(int_mtx, 2, mean), '-')  %>% # column wise sweeping
    sweep(2, apply(int_mtx, 2, sd), '/')
head(int_z, 2)

,sp|A0AVT1|UBA6_HUMAN,sp|O00170|AIP_HUMAN,sp|O00299|CLIC1_HUMAN,sp|O00429|DNM1L_HUMAN,sp|O00483|NDUA4_HUMAN,sp|O00571|DDX3X_HUMAN,sp|O14828|SCAM3_HUMAN,sp|O14929|HAT1_HUMAN,sp|O15143|ARC1B_HUMAN,sp|O15145|ARPC3_HUMAN,...,sp|Q9Y230|RUVB2_HUMAN,sp|Q9Y285|SYFA_HUMAN,sp|Q9Y2B0|CNPY2_HUMAN,sp|Q9Y2X3|NOP58_HUMAN,sp|Q9Y383|LC7L2_HUMAN,sp|Q9Y4L1|HYOU1_HUMAN,sp|Q9Y5K5|UCHL5_HUMAN,sp|Q9Y5M8|SRPRB_HUMAN,sp|Q9Y5X3|SNX5_HUMAN,sp|Q9Y678|COPG1_HUMAN
CTG-0158,0.2093806,0.65302273,-1.0997903,-0.6676739,-0.4647837,-1.1040398,1.064904,1.2123033,-0.1051954,0.2626795,...,-0.2250597,-0.5820609,0.1906079,0.5237176,0.9304869,1.9736444,-0.1122720,-0.27387703,0.4041529,0.4389799
CTG-0159,-0.1839989,-0.01878222,-0.3120364,-0.9488586,-1.0432424,-0.5989767,-2.682560,-0.3098403,0.3678152,0.5056474,...,0.7999883,-0.2509652,0.1577792,0.4496930,0.3787295,-0.7937075,0.3066905,0.09551761,-0.8271938,0.8381110


Save the the data here, so we do not need to generate it every time.

## Create a table with missing values

In [22]:
int_mis <- prodNA(int_z, noNA = 0.1)

In [26]:
# summary(int_mis)

In [28]:
min(unlist(int_mis), na.rm = TRUE)

[1] -12.07765

In [29]:
max(unlist(int_mis), na.rm = TRUE)

[1] 5.838831

## Imputation - random forest

Notice the function assumes that the columns are features and rows are samples.

In [23]:
imp_rf <- missForest(int_mis)

  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 3 in progress...done!
  missForest iteration 4 in progress...done!


In [24]:
imp_rf$OOBerror

NRMSE 
0.601409

In [ ]:
# tictoc::tic("RF")
# tmp <- missForest(ion_miss[[i]])
# tmp2 <- toc()
# time.table$RF[i] <- as.numeric(tmp2$toc - tmp2$tic)

## Imputation - kNN

In [33]:
imp_knn <- kNN(int_mis, k = 6)

Calculate NRMSE for kNN and compare with missForest (with 10 rounds of random removal of values).